In [5]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['text.usetex'] = True
import os
import csv
from numpy import savetxt

In [6]:
FrcDat = pd.read_csv("FractureData.csv")

#Define necessary Dictionary items to get data from force-disp raw file
dict_it = tuple(['SetName','Cycle','Time_S','Size_mm','Displacement_mm','Force_N'])
Type = tuple(['Stretch'])
Cycle = tuple(['1-Preload','1-Stretch'])



In [7]:
FrcDat

,Name,Pig,Location,Direction,Test,Width,Thickness
0,PA_Belly_SubQ1_frac,A,Belly,Parallel,DF,31.416,1.910
1,PA_Belly_SubQ2_frac,A,Belly,Parallel,F,28.122,1.856
2,PA_Belly_SubQ4_frac,A,Belly,Perpendicular,DF,33.514,1.528
3,PA_Belly_SubQ5_frac,A,Belly,Perpendicular,DF,29.352,2.192
4,PA_Belly_SubQ6_frac,A,Belly,Parallel,DF,29.514,1.744
5,PA_Belly_SubQ8_frac,A,Belly,Perpendicular,DF,28.452,1.240
6,PA_Breast_SubQ1_frac,A,Breast,Parallel,DF,33.316,1.262
7,PA_Breast_SubQ2_frac,A,Breast,Parallel,F,19.460,1.806
8,PA_Breast_SubQ3_frac,A,Breast,Perpendicular,DF,31.198,1.286
9,PA_Breast_SubQ5_frac,A,Breast,Parallel,DF,25.272,1.172


In [8]:
for j in range(0,len(FrcDat)):
    Specimen = FrcDat['Name'][j]
    Filename  = Specimen+'Data.csv' #File name
    thickness = FrcDat['Thickness'][j]   # Average thickness in mm (from measurements)
    width = FrcDat['Width'][j]       # Average width in mm
   

    print(f"Processing {Specimen}")
        
    fig,ax  = plt.subplots(1,sharey=False,dpi=300)

    i = 0
    Time = []
    Size = []
    Force = []
    Force = []

    for i in range(0,len(Type)):
        T_Type = Type[i]
        with open(Filename, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row[dict_it[0]]==T_Type:
                    if row[dict_it[1]]!=Cycle[0]:
                        Time.append(float(row[dict_it[2]])) #in seconds
                        Size.append(float(row[dict_it[3]])) #in mm
                        Force.append(float(row[dict_it[5]]))# in Newtons

    try:
        os.mkdir(Specimen)
    except FileExistsError:
        pass

    Xlam = np.asarray(Size)/Size[0]
    Ylam = 1/Xlam
    Cauchy_stress  = Force/(width*thickness*Ylam)
    ax.plot(Xlam,Cauchy_stress,lw=2.0)
    plt.ylabel(r'$\sigma$ [MPa]',fontsize=18)
    plt.xlabel(r'$\lambda$ [-]',fontsize=18)
    plt.savefig(fname = Specimen+'/Stress_Stretch.png',dpi=300)
    plt.close(fig)

    for i in range(0,len(Type)):
        T_Type = Type[i]
        Time = []
        Size = []
        Force = []
        Force = []
        with open(Filename, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row[dict_it[0]]==T_Type:
                    if row[dict_it[1]]!=Cycle[0] and row[dict_it[1]]==Cycle[1] :
                        Time.append(float(row[dict_it[2]])) #in seconds
                        Size.append(float(row[dict_it[3]])) #in mm
                        Force.append(float(row[dict_it[5]]))# in Newtons
        Xlam = np.asarray(Size)/Size[0]
        Ylam = 1/Xlam
        Cauchy_stress  = Force/(width*thickness*Ylam)
        savetxt(Specimen+'/'+str(i+1)+'.csv',np.transpose(np.asarray([Xlam,Cauchy_stress])),header='Stretch,Cauchy_stress', delimiter=',')


Processing PA_Belly_SubQ1_frac
Processing PA_Belly_SubQ2_frac
Processing PA_Belly_SubQ4_frac
Processing PA_Belly_SubQ5_frac
Processing PA_Belly_SubQ6_frac
Processing PA_Belly_SubQ8_frac
Processing PA_Breast_SubQ1_frac
Processing PA_Breast_SubQ2_frac
Processing PA_Breast_SubQ3_frac
Processing PA_Breast_SubQ5_frac
Processing PA_Breast_SubQ6_frac
Processing PA_Breast_SubQ7_frac1
Processing PB_Belly_SubQ1_frac
Processing PB_Belly_SubQ2_frac
Processing PB_Belly_SubQ3_frac
Processing PB_Belly_SubQ4_frac
Processing PB_Belly_SubQ5_frac
Processing PB_Belly_SubQ6_frac
Processing PB_Belly_SubQ7_frac
Processing PB_Belly_SubQ8_frac
Processing PB_Breast_SubQ2_frac
Processing PB_Breast_SubQ3_frac
Processing PB_Breast_SubQ6_frac
Processing PC_Belly_SubQ2_Frac
Processing PC_Belly_SubQ5_Frac
Processing PC_Belly_SubQ6_Frac
Processing PC_Belly_SubQ7_Frac
Processing PC_Breast_SubQ3_Frac
Processing PC_Breast_SubQ6_Fracture
Processing PC_Breast_SubQ7_Fracture
Processing Pig3_Belly_SubQ1_frac
Processing Pig3_